<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%8B%A4%EA%B8%B0_Part2_Chapter04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [29]:
display(train.head(3))

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,NCR06,12.5,Low Fat,0.006760,Household,42.8112,OUT013,1987,High,Tier 3,Supermarket Type1,639.1680
1,FDW11,12.6,Low Fat,0.048741,Breads,60.4194,OUT013,1987,High,Tier 3,Supermarket Type1,990.7104
2,FDH32,12.8,Low Fat,0.075997,Fruits and Vegetables,97.1410,OUT013,1987,High,Tier 3,Supermarket Type1,2799.6890


#데이터 전처리 - 인코딩

In [30]:
#인코딩할 컬럼 선택
print(list(train.columns[train.dtypes == 'object']))

cols = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
print(cols)

['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']


In [31]:
#target 데이터 따로 빼두기
target = train.pop('Item_Outlet_Sales')
print(train.shape, test.shape)

(6818, 11) (1705, 11)


In [32]:
#인코딩 : Label Encoder 적용
data = pd.concat([train, test], axis=0)
print(data.shape)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in cols:
  data[col] = le.fit_transform(data[col])

data.head(3)

(8523, 11)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,NCR06,12.5,1,0.006760,9,42.8112,1,1987,0,2,1
1,FDW11,12.6,1,0.048741,1,60.4194,1,1987,0,2,1
2,FDH32,12.8,1,0.075997,6,97.1410,1,1987,0,2,1


In [33]:
#인코딩 : cat.codes 적용
data = pd.concat([train, test], axis=0)
print(data.shape)

for col in cols:
  data[col] = data[col].astype('category').cat.codes

data.head(3)

(8523, 11)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,NCR06,12.5,1,0.006760,9,42.8112,1,1987,0,2,1
1,FDW11,12.6,1,0.048741,1,60.4194,1,1987,0,2,1
2,FDH32,12.8,1,0.075997,6,97.1410,1,1987,0,2,1


In [34]:
#인코딩 후 train, test 재분배
train = data.iloc[:len(train), ].copy()
test = data.iloc[len(train):, ].copy()

#데이터 전처리 - 결측치 처리

In [35]:
train.isnull().sum() #Item Weight & Outlet_size(이건 결측치 없는데 왜함?)

IW_min = train['Item_Weight'].min()
OS_mode = train['Outlet_Size'].mode()[0]
print(IW_min, OS_mode)

train['Item_Weight'] = train['Item_Weight'].fillna(IW_min)
train['Outlet_Size'] = train['Outlet_Size'].fillna(OS_mode)
test['Item_Weight'] = test['Item_Weight'].fillna(IW_min)
test['Outlet_Size'] = test['Outlet_Size'].fillna(OS_mode)

4.555 1


#데이터 전처리 - 불필요한 칼럼 삭제

In [36]:
print(train.shape, test.shape)
train.drop('Item_Identifier', axis=1, inplace=True)
test.drop('Item_Identifier', axis=1, inplace=True)
print(train.shape, test.shape)

(6818, 11) (1705, 11)
(6818, 10) (1705, 10)


#검증 데이터 나누기

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state = 42)

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(5454, 10) (1364, 10) (5454,) (1364,)


#모델 학습/평가

In [40]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import root_mean_squared_error

In [41]:
def print_metrics(y_val, y_pred):
  mse = mean_squared_error(y_val, y_pred)
  mae = mean_absolute_error(y_val, y_pred)
  r2 = r2_score(y_val, y_pred)
  rmse = root_mean_squared_error(y_val, y_pred)

  print('MSE : ',mse)
  print('MAE : ',mae)
  print('R2 score : ', r2)
  print('RMSE : ', rmse)


In [42]:
#선형 회귀
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)

print_metrics(y_val, y_pred)

MSE :  1407084.930480854
MAE :  882.3529877260387
R2 score :  0.5288442131039772
RMSE :  1186.20610792596


In [44]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)

print_metrics(y_val, y_pred)


MSE :  1164103.8767415415
MAE :  754.4460031510264
R2 score :  0.6102052788757837
RMSE :  1078.9364563038648


In [46]:
from lightgbm import LGBMRegressor
model = LGBMRegressor(random_state = 99, verbose=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

print_metrics(y_val, y_pred)

MSE :  1166040.259755914
MAE :  750.1547119219582
R2 score :  0.6095568901089756
RMSE :  1079.8334407471896


#예측/결과 파일 생성

In [47]:
pred = model.predict(test)
pred

array([1302.58155145,  816.75188984, 1919.61477245, ..., 4085.53462894,
        913.59532252, 1306.34989874])

In [48]:
submit = pd.DataFrame({'pred' : pred})
submit

,pred
0,1302.581551
1,816.751890
2,1919.614772
3,1378.068136
4,2743.990658
...,...
1700,261.821356
1701,805.268855
1702,4085.534629
1703,913.595323


In [49]:
submit.to_csv('result.csv', index=False)

In [51]:
pd.read_csv('result.csv')

,pred
0,1302.581551
1,816.751890
2,1919.614772
3,1378.068136
4,2743.990658
...,...
1700,261.821356
1701,805.268855
1702,4085.534629
1703,913.595323
